In [1]:
'''Clean up trip data files into unified dataset

'''

# %% Initialize
import glob
import pandas as pd
import numpy as np

path = 'data/'


In [2]:
# Load Hubway Trips CSVs
baseName = 'hubway_Trips_'

files = glob.glob(f'{path}{baseName}*.csv')
data_hubwaytrips = pd.DataFrame()
for f in files:
    df = pd.read_csv(f,
                     dtype = {'Duration': int,
                              'Start date': str,
                              'End date': str,
                              'Start station number': str,
                              'Start station name': str,
                              'End station number': str,
                              'End station name': str,
                              'Bike number': str,
                              'Member type': str,
                              'Zip code': str,
                              'Gender': str},)
                    # parse_dates=['Start date', 'End date'],)

    data_hubwaytrips = pd.concat([data_hubwaytrips, df])
# print(data_hubwaytrips.columns.values)

# Clean up for merging
data_hubwaytrips.rename(columns={'Duration': 'duration',
                                'Start date': 'start_time',
                                'End date': 'end_time',
                                'Start station number': 'start_station_id',
                                'Start station name': 'start_station_name',
                                'End station number': 'end_station_id',
                                'End station name': 'end_station_name',
                                'Bike number': 'bike_id',
                                'Member type': 'user_type',
                                'Zip code': 'zip_code',
                                'Gender': 'gender'},
                        inplace=True)

datefmt = '%m/%d/%Y %H:%M'
data_hubwaytrips['start_time'] = pd.to_datetime(data_hubwaytrips['start_time'], format=datefmt)
data_hubwaytrips['end_time'] = pd.to_datetime(data_hubwaytrips['end_time'], format=datefmt)

# Gender - 0=male; 1=female
data_hubwaytrips['gender'].replace('Male', 0, inplace=True)
data_hubwaytrips['gender'].replace('Female', 1, inplace=True)
data_hubwaytrips['gender'].replace('NaN', np.nan, inplace=True)

# User Type - 1=Member, 0=Other
data_hubwaytrips['user_type'].replace('Member', 1, inplace=True)
data_hubwaytrips['user_type'].replace('Casual', 0, inplace=True)
data_hubwaytrips['user_type'].replace('Subscriber', 1, inplace=True)
data_hubwaytrips['user_type'].replace('Customer', 0, inplace=True)

data_hubwaytrips.to_pickle(f'{path}hubway_Trips.pkl')
# print(data_hubwaytrips.columns.values)
# print(data_hubwaytrips.dtypes)

In [10]:
# Load Hubway Trtipdata CSVs
baseName = '?hubway?tripdata.csv'

files = glob.glob(f'{path}*{baseName}')

data_hubway = pd.DataFrame()
for f in files:
    df = pd.read_csv(f,
                     dtype = {'tripduration': int,
                              'starttime': str,
                              'stoptime': str,
                              'start station id': str,
                              'start station name': str,
                              'start station latitude': str,
                              'start station longitude': str,
                              'end station id': str,
                              'end station name': str,
                              'end station latitude': str,
                              'end station longitude': str,
                              'bikeid': str,
                              'usertype': str,
                              'birth year': str,
                              'gender': int},)

    data_hubway = pd.concat([data_hubway, df])

# print(data_hubway.columns.values)

# Clean up for merging
data_hubway.rename(columns={'tripduration': 'duration',
                            'starttime': 'start_time',
                            'stoptime': 'end_time',
                            'start station id': 'start_station_id',
                            'start station name': 'start_station_name',
                            'start station latitude': 'start_station_latitude',
                            'start station longitude': 'start_station_longitude',
                            'end station id': 'end_station_id',
                            'end station name': 'end_station_name',
                            'end station latitude': 'end_station_latitude',
                            'end station longitude': 'end_station_longitude',
                            'bikeid': 'bike_id',
                            'usertype': 'user_type',
                            'birth year': 'birth_year',
                            'gender': 'gender'},
                        inplace=True)

datefmt = '%Y-%m-%d %H:%M:%S'
data_hubway['start_time'] = pd.to_datetime(data_hubway['start_time'], format=datefmt)
data_hubway['end_time'] = pd.to_datetime(data_hubway['end_time'], format=datefmt)

# Gender - 0=male; 1=female
data_hubway['gender'].replace(0, np.nan, inplace=True)
data_hubway['gender'].replace(1, 0, inplace=True)
data_hubway['gender'].replace(2, 1, inplace=True)
data_hubway['gender'].replace('Male', 0, inplace=True)
data_hubway['gender'].replace('Female', 1, inplace=True)

# User Type - 1=Member, 0=Other
data_hubway['user_type'].replace('Subscriber', 1, inplace=True)
data_hubway['user_type'].replace('Customer', 0, inplace=True)
data_hubway['user_type'].replace('Member', 1, inplace=True)
data_hubway['user_type'].replace('Casual', 0, inplace=True)

data_hubway['birth_year'].replace(r'\N', np.nan, inplace=True)
data_hubway['start_station_latitude'].replace(r'\N', np.nan, inplace=True)
data_hubway['start_station_longitude'].replace(r'\N', np.nan, inplace=True)
data_hubway['end_station_latitude'].replace(r'\N', np.nan, inplace=True)
data_hubway['end_station_longitude'].replace(r'\N', np.nan, inplace=True)

toNumeric = ['birth_year', 'start_station_latitude', 'start_station_longitude','end_station_latitude', 'end_station_longitude']
data_hubway[toNumeric] = data_hubway[toNumeric].apply(pd.to_numeric)

data_hubway.to_pickle(f'{path}hubway-tripdata.pkl')
# print(data_hubway.columns.values)
# print(data_hubway.dtypes)

In [20]:
# Get Bluebike CSV column counts
# Determine when format changes happened
baseName = '-bluebikes-tripdata.csv'

files = glob.glob(f'{path}*{baseName}')

lastCols = []
changefmt = []

for i, f in enumerate(files):
    df = pd.read_csv(f)
    # print(f'{i} - {f}: {len(df.columns)}')

    if lastCols != list(df.columns):
        # print(i)
        changefmt.append(i)

    lastCols = list(df.columns)

changefmt.append(i)

print(f'{changefmt=}')


changefmt=[0, 24, 59, 62]


In [7]:
# Load Bluebike CSVs
baseName = '-bluebikes-tripdata.csv'

files = glob.glob(f'{path}*{baseName}')
# print(len(files))

changefmt=[0, 24, 59, 62]

dtypes1 =  {'tripduration': int,
            'starttime': str,
            'stoptime': str,
            'start station id': str,
            'start station name': str,
            'start station latitude': str,
            'start station longitude': str,
            'end station id': str,
            'end station name': str,
            'end station latitude': str,
            'end station longitude': str,
            'bikeid': str,
            'usertype': str,
            'birth year': str,
            'gender': int}

dtypes2 =  {'tripduration': int,
            'starttime': str,
            'stoptime': str,
            'start station id': str,
            'start station name': str,
            'start station latitude': str,
            'start station longitude': str,
            'end station id': str,
            'end station name': str,
            'end station latitude': str,
            'end station longitude': str,
            'bikeid': str,
            'usertype': str,
            'postal code': str}

dtypes3 =  {'ride_id': str,
            'rideable_type': str,
            'tripduration': int,
            'started_at': str,
            'ended_at': str,
            'start_station_name': str,
            'start_station_id': str,
            'end_station_name': str,
            'end_station_id': str,
            'start_lat': str,
            'start_lng': str,
            'end_lat': str,
            'end_lng': str,
            'member_casual': str}

dtypes = [dtypes1, dtypes2, dtypes3]

cols1 = {'tripduration': 'duration',
        'starttime': 'start_time',
        'stoptime': 'end_time',
        'start station id': 'start_station_id',
        'start station name': 'start_station_name',
        'start station latitude': 'start_station_latitude',
        'start station longitude': 'start_station_longitude',
        'end station id': 'end_station_id',
        'end station name': 'end_station_name',
        'end station latitude': 'end_station_latitude',
        'end station longitude': 'end_station_longitude',
        'bikeid': 'bike_id',
        'usertype': 'user_type',
        'birth year': 'birth_year',
        'gender': 'gender'}

cols2 = {'tripduration': 'duration',
        'starttime': 'start_time',
        'stoptime': 'end_time',
        'start station id': 'start_station_id',
        'start station name': 'start_station_name',
        'start station latitude': 'start_station_latitude',
        'start station longitude': 'start_station_longitude',
        'end station id': 'end_station_id',
        'end station name': 'end_station_name',
        'end station latitude': 'end_station_latitude',
        'end station longitude': 'end_station_longitude',
        'bikeid': 'bike_id',
        'usertype': 'user_type',
        'postal code': 'zip_code'}

cols3 = {'ride_id': 'ride_id',
         'rideable_type': 'rideable_type',
         'tripduration': 'duration',
         'started_at': 'start_time',
         'ended_at': 'end_time',
         'start_station_name': 'start_station_name',
         'start_station_id': 'start_station_id',
         'end_station_name': 'end_station_name',
         'end_station_id': 'end_station_id',
         'start_lat': 'start_station_latitude',
         'start_lng': 'start_station_longitude',
         'end_lat': 'end_station_latitude',
         'end_lng': 'end_station_longitude',
         'member_casual': 'user_type'}

cols=[cols1, cols2, cols3]

for x in range(len(changefmt)-1):
    # print(x)
    s = changefmt[x]
    e = changefmt[x+1]

    data_BB = pd.DataFrame()
    for f in files[s:e]:
        print(f)
        df = pd.read_csv(f, dtype=dtypes[x])
        data_BB = pd.concat([data_BB, df], ignore_index=True)

    data_BB.rename(columns=cols[x],
                        inplace=True)
    
    datefmt = '%Y-%m-%d %H:%M:%S'
    data_BB['start_time'] = pd.to_datetime(data_BB['start_time'], format=datefmt)
    data_BB['end_time'] = pd.to_datetime(data_BB['end_time'], format=datefmt)

    # User Type - 1=Member, 0=Other
    data_BB['user_type'].replace('Subscriber', 1, inplace=True)
    data_BB['user_type'].replace('Customer', 0, inplace=True)
    data_BB['user_type'].replace('member', 1, inplace=True)
    data_BB['user_type'].replace('casual', 0, inplace=True)
    data_BB['user_type'].replace('Member', 1, inplace=True)
    data_BB['user_type'].replace('Casual', 0, inplace=True)

    if x == 0:
        # Gender - 0=male; 1=female
        data_BB['gender'].replace(0, np.nan, inplace=True)
        data_BB['gender'].replace(1, 0, inplace=True)
        data_BB['gender'].replace(2, 1, inplace=True)
        data_BB['gender'].replace('Male', 0, inplace=True)
        data_BB['gender'].replace('Female', 1, inplace=True)
        

        toNumeric = ['birth_year']
        data_BB[toNumeric] = data_BB[toNumeric].apply(pd.to_numeric)

    toNumeric = ['start_station_latitude', 'start_station_longitude','end_station_latitude', 'end_station_longitude']
    data_BB[toNumeric] = data_BB[toNumeric].apply(pd.to_numeric)

    data_BB.to_pickle(f'{path}bluebike-tripdata-{x+1}.pkl')
    # print(data_BB.columns.values)

data_BB1 = pd.read_pickle(f'{path}bluebike-tripdata-1.pkl')
data_BB2 = pd.read_pickle(f'{path}bluebike-tripdata-2.pkl')
data_BB3 = pd.read_pickle(f'{path}bluebike-tripdata-3.pkl')

data\201805-bluebikes-tripdata.csv
data\201806-bluebikes-tripdata.csv
data\201807-bluebikes-tripdata.csv
data\201808-bluebikes-tripdata.csv
data\201809-bluebikes-tripdata.csv
data\201810-bluebikes-tripdata.csv
data\201811-bluebikes-tripdata.csv
data\201812-bluebikes-tripdata.csv
data\201901-bluebikes-tripdata.csv
data\201902-bluebikes-tripdata.csv
data\201903-bluebikes-tripdata.csv
data\201904-bluebikes-tripdata.csv
data\201905-bluebikes-tripdata.csv
data\201906-bluebikes-tripdata.csv
data\201907-bluebikes-tripdata.csv
data\201908-bluebikes-tripdata.csv
data\201909-bluebikes-tripdata.csv
data\201910-bluebikes-tripdata.csv
data\201911-bluebikes-tripdata.csv
data\201912-bluebikes-tripdata.csv
data\202001-bluebikes-tripdata.csv
data\202002-bluebikes-tripdata.csv
data\202003-bluebikes-tripdata.csv
data\202004-bluebikes-tripdata.csv
data\202005-bluebikes-tripdata.csv
data\202006-bluebikes-tripdata.csv
data\202007-bluebikes-tripdata.csv
data\202008-bluebikes-tripdata.csv
data\202009-bluebike

In [11]:
# Merge data
data_BB1 = pd.read_pickle(f'{path}bluebike-tripdata-1.pkl')
data_BB2 = pd.read_pickle(f'{path}bluebike-tripdata-2.pkl')
data_BB3 = pd.read_pickle(f'{path}bluebike-tripdata-3.pkl')
data_HW1 = pd.read_pickle(f'{path}hubway_Trips.pkl')
data_HW2 = pd.read_pickle(f'{path}hubway-tripdata.pkl')

data = pd.concat([data_BB1, data_BB2, data_BB3, data_HW1, data_HW2],
                 ignore_index=True)

data.sort_values(by=['start_time'], ignore_index=True, inplace=True)

data.to_pickle(f'{path}all_data.pkl')

In [19]:
# totalsYearly = data['start_time'].groupby(data['start_time'].dt.year).count()

totalsMonthly = data['start_time'].groupby([data['start_time'].dt.year, data['start_time'].dt.month]).count().rename_axis(['year','month']).reset_index()
totalsMonthly.to_csv('data/totalMonthly.csv')

In [6]:
# Read data
# data = pd.read_pickle(f'{path}all_data.pkl')